In [1]:
!pip install mysql-connector

In [3]:
import mysql.connector
import datetime
from datetime import datetime, timedelta
mysql_config = {
    'user': 'root',
    'password': 'Nb12345678',
    'host': 'localhost',
    'database': 'ranbooks'
}
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

def add_books():
    while True:
        code_input = input("Enter Book Code: ")
        try:
            code = int(code_input)
            break  
        except ValueError:
            print("Invalid input. Please enter a valid integer for the book code.")

    title = input("Enter Book Title: ")
    author = input("Enter Book Author: ")
    year = int(input("Enter Book publication year: "))
    quantity = int(input("Enter Book Quantity: "))
    cursor.execute("INSERT INTO books (Code, Title, Author, Year, Quantity) VALUES (%s, %s, %s, %s, %s)", (code, title, author, year, quantity))
    conn.commit()
    print("Record ADDED successfully...")


def display_books():
    cursor.execute("SELECT * FROM books")
    books = cursor.fetchall()
    if books:
     print("Books in RAN Library :")
    for book in books:
        print(f"Code: {book[0]}, Title: {book[1]}, Author: {book[2]}, Year: {book[3]}, Quantity: {book[4]}")


def search_books():
    op = input("You want to search about book by 1-Code 2-Title 3-Author name? ")
    search_term = input("Enter search term: ")
    cursor.execute(f"SELECT * FROM books WHERE {'Code' if op == '1' else 'Title' if op == '2' else 'Author'} LIKE %s", (f'%{search_term}%',))
    search_results = cursor.fetchall()
    if search_results:
        print("Search results:")
        for book in search_results:
            print(f"Code: {book[0]}, Title: {book[1]}, Author: {book[2]}, Year: {book[3]}, Quantity: {book[4]}")
    else:
        print("Record NOT found..!!!")

def delete_books():
    c = input("You want to delete all the books or a specific one? Enter 1 for all or 2 for a specific one: ")
    if c == "1":
        a = input("Are you SURE to DELETE all the books (Y/N): ")
        if a.lower() == "y":
            cursor.execute("DELETE FROM books")
            conn.commit()
            print("All books deleted successfully...")
        else:
            print("Books NOT deleted.")
    elif c == "2":
        sp = input("Please enter the code for the book you want to delete: ")
        cursor.execute("SELECT * FROM books WHERE Code = %s", (sp,))
        book = cursor.fetchone()
        if book:
            cursor.execute("DELETE FROM books WHERE Code = %s", (sp,))
            conn.commit()
            print("Book deleted successfully.")
        else:
            print("Book with the given code not found.")
    else:
        print("Invalid input. Please enter 1 or 2.")

                       
def update_booksinfo():
    code = int(input("Enter Book Code to update: "))
    new_title = input("Enter new Title: ")
    new_author = input("Enter new Author: ")
    new_year = int(input("Enter new publication year: "))
    new_quantity = int(input("Enter new Quantity: "))
    
    cursor.execute("UPDATE books SET Title=%s, Author=%s, Year=%s, Quantity=%s WHERE Code=%s",
                   (new_title, new_author, new_year, new_quantity, code))
    conn.commit()
    print("Record UPDATED successfully...")
    
def borrow_book(code, name, conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books WHERE Code = %s", (code,))
    book = cursor.fetchone()
    
    if book:
        borrow_date = datetime.now().date()  
        return_date = borrow_date + timedelta(days=5)  

        cursor.execute("INSERT INTO borrow_book (Code, Title, Author, Name, borrow_date, return_date) VALUES (%s, %s, %s, %s, %s, %s)",
                       (book[0], book[1], book[2], name, borrow_date, return_date))
        conn.commit()

        cursor.execute("UPDATE books SET Quantity = Quantity - 1 WHERE Code = %s", (code,))
        conn.commit()

        print("Book borrowed successfully.")
    else:
        print("Book not found in the library.")

    cursor.close()

def return_book(code,name,conn):
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT * FROM borrow_book WHERE Code = %s", (code,))
    borrow_book = cursor.fetchone()

    if borrow_book:
        return_time = datetime.now()
        cursor.execute("UPDATE borrow_book SET return_date = %s WHERE Code = %s",
                       (return_time, code))
        conn.commit()

        cursor.execute("UPDATE books SET Quantity = Quantity + 1 WHERE Code = %s", (code,))
        conn.commit()

        print(f"Book '{borrow_book['Title']}' returned successfully.")
    else:
        print("Book with the given code is not currently borrowed.")
        
def display_borrowed_books(conn):
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT Code, Name, Title, Author, Borrow_Date, Return_Date FROM borrow_book")
    borrow_book = cursor.fetchall()

    if borrow_book:
        print("Borrowed Books:")
        for book in borrow_book:
            print(f"Code: {book['Code']}, Name: {book['Name']}, Title: {book['Title']}, Author: {book['Author']}, Borrow Date: {book['Borrow_Date']}, Return Date: {book['Return_Date']} ")
    else:
        print("No books currently borrowed.")

def move_book():
    book_code = input("Enter the code of the book you want to move: ")
    
    cursor.execute("SELECT * FROM books WHERE Code = %s", (book_code,))
    book = cursor.fetchone()
    
    if book:
        cursor.execute("INSERT INTO neglected_books (Code, Title, Author, Year, Quantity) VALUES (%s, %s, %s, %s, %s)",
                       (book[0], book[1], book[2], book[3], book[4]))
        conn.commit()
        cursor.execute("DELETE FROM books WHERE Code = %s", (book_code,))
        conn.commit()
        
        print("Book moved to neglected table successfully.")
    else:
        print("Book with the given code not found in the library.")

def display_neglected_books():
    cursor.execute("SELECT * FROM neglected_books")
    neglected_books = cursor.fetchall()
    if neglected_books:
        print("Books in the neglected table:")
        for book in neglected_books:
            print(f"Code: {book[0]}, Title: {book[1]}, Author: {book[2]}, Year: {book[3]}, Quantity: {book[4]}")

      
while True:
    print("\n RAN LIBRARY BOOKS :\n")
    print("==================================")
    print("1. Add New Book record")
    print("2. Display Book record")
    print("3. Search Book record")
    print("4. Delete Book records")
    print("5. Borrow Book")
    print("6. Return Book")
    print("7. Update Book record")
    print("8. Move Book")
    print("9. Display Neglected Book List") 
    print("10.display borrowed books List")
    print("11. EXIT")
    print("==================================")
    choice = input("\nEnter choice (1-11): ")
    if choice == "1":
        add_books()
    elif choice == "2":
        display_books()
    elif choice == "3":
        search_books()
    elif choice == "4":
        delete_books()
    elif choice == "5":
        code = input("Enter the code of the book you want to borrow: ")
        name = input("Enter the borrower name: ")
        borrow_book(code, name,conn)
    elif choice == "6":
        code = input("Enter the code of the book you want to return: ")
        return_book(code,name,conn)
    elif choice == "7":
        update_booksinfo()
    elif choice == "8":
        move_book()
    elif choice == "9":
        display_neglected_books()
    elif choice == "10":
        display_borrowed_books(conn)   
    elif choice == "11":
        print("Thanks, Have a nice day :) ")
        break
    else:
        print("Invalid choice. Please enter choice 1-8.")


 RAN LIBRARY BOOKS :

1. Add New Book record
2. Display Book record
3. Search Book record
4. Delete Book records
5. Borrow Book
6. Return Book
7. Update Book record
8. Move Book
9. Display Neglected Book List
10.display borrowed books List
11. EXIT

Enter choice (1-11): 2
Books in RAN Library :
Code: 112, Title: The Art of Computer Programming, Author: Donald Knuth, Year: 1986, Quantity: 40
Code: 113, Title: Introduction to the Theory of Computation, Author: Michael Sipser (, Year: 1996, Quantity: 55
Code: 114, Title: Algorithms, Author: Tom, Year: 1797, Quantity: 40
Code: 115, Title: You Don't Know, Author: Kiyle, Year: 1999, Quantity: 55
Code: 116, Title: Matilda, Author: Roald, Year: 1980, Quantity: 20
Code: 117, Title: Alice's Adventures in Wonderland, Author: Lawies, Year: 1865, Quantity: 66

 RAN LIBRARY BOOKS :

1. Add New Book record
2. Display Book record
3. Search Book record
4. Delete Book records
5. Borrow Book
6. Return Book
7. Update Book record
8. Move Book
9. Display N